<a href="https://colab.research.google.com/github/hasse910/IA_CRE/blob/main/IA_Cresol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!echo "Versão do Ollama após a instalação:"
!ollama --version

Versão do Ollama após a instalação:


In [ ]:
!curl --version
!python3 --version
!pip3 --version

!pip install ollama langchain langchain-community langchain-ollama faiss-cpu

!sudo apt-get update
!sudo apt-get install -y pciutils
!sudo apt-get install -y lspci

!curl -fsSL https://ollama.com/install.sh | sh

!useradd -r -s /bin/false -m -d /usr/share/ollama ollama
!groupadd ollama # Pode já existir
!usermod -a -G ollama $(whoami) # Adiciona seu usuário atual ao grupo ollama
!chown -R ollama:ollama /usr/share/ollama # Garante permissões no diretório home do usuário ollama

#executa em segundo plano:
!nohup ollama serve > /dev/null 2>&1 &

curl 7.81.0 (x86_64-pc-linux-gnu) libcurl/7.81.0 OpenSSL/3.0.2 zlib/1.2.11 brotli/1.0.9 zstd/1.4.8 libidn2/2.3.2 libpsl/0.21.0 (+libidn2/2.3.2) libssh/0.9.6/openssl/zlib nghttp2/1.43.0 librtmp/2.3 OpenLDAP/2.5.17
Release-Date: 2022-01-05
Protocols: dict file ftp ftps gopher gophers http https imap imaps ldap ldaps mqtt pop3 pop3s rtmp rtsp scp sftp smb smbs smtp smtps telnet tftp 
Features: alt-svc AsynchDNS brotli GSS-API HSTS HTTP2 HTTPS-proxy IDN IPv6 Kerberos Largefile libz NTLM NTLM_WB PSL SPNEGO SSL TLS-SRP UnixSockets zstd
Python 3.11.13
pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/ubuntugis

In [ ]:
!ollama pull gemma3:4b
!ollama pull gemma3:12b
!ollama pull nomic-embed-text

In [ ]:
!ollama list

NAME                       ID              SIZE      MODIFIED       
nomic-embed-text:latest    0a109f422b47    274 MB    23 seconds ago    
gemma3:12b                 f4031aab637d    8.1 GB    23 seconds ago    
gemma3:4b                  a2af6cc3eb7f    3.3 GB    4 minutes ago     


In [ ]:
import ollama
import sys
import os
import shutil # remover diretórios (como o índice FAISS)
import pickle # Para salvar/carregar o timestamp do documento
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaLLM

# --- Configuração ---
# Certifique-se que este arquivo existe no mesmo diretório ou forneça o caminho completo
DOCUMENT_PATH = "./procedures.txt"
CHUNK_SIZE = 750
CHUNK_OVERLAP = 100
EMBEDDING_MODEL_NAME = "nomic-embed-text"
LLM_MODEL_NAME = "gemma3:4b"
FAISS_INDEX_PATH = "faiss_index_procedures"
# Caminho para o arquivo que guardará o timestamp do documento quando o índice foi criado
INDEX_METADATA_PATH = os.path.join(FAISS_INDEX_PATH, "index_metadata.pkl")

def run_rag_query():
    # --- Inicialização dos Embeddings (movido para cima para estar disponível para FAISS.load_local) ---
    print(f"Inicializando embeddings Ollama com o modelo: {EMBEDDING_MODEL_NAME}...")
    try:
        embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
        _ = embeddings.embed_query("Teste de conexão com embedding.")
        print("Embeddings Ollama inicializados com sucesso.")
    except Exception as e:
        print(f"\nErro ao inicializar embeddings Ollama: {e}")
        print(f"Verifique se o serviço Ollama está rodando e se o modelo '{EMBEDDING_MODEL_NAME}' foi baixado.")
        sys.exit(1)

    vectorstore = None
    recreate_index = True #Assume que precisa recriar por padrão

    # --- VERIFICAÇÃO DO ÍNDICE EXISTENTE E DO TIMESTAMP DO DOCUMENTO ---
    if os.path.exists(FAISS_INDEX_PATH) and os.path.exists(INDEX_METADATA_PATH):
        try:
            print(f"Verificando metadados do índice em {INDEX_METADATA_PATH}...")
            with open(INDEX_METADATA_PATH, 'rb') as f:
                metadata = pickle.load(f)
            saved_doc_timestamp = metadata.get('doc_timestamp')

            if not os.path.exists(DOCUMENT_PATH):
                print(f"Erro: O arquivo de documento original {DOCUMENT_PATH} não foi encontrado. Não é possível verificar se o índice está atualizado.")
                print("Você pode precisar apagar o diretório do índice manualmente se o caminho do documento mudou.")
                # Decide-se carregar o índice existente mesmo assim, ou sair.
                # Por segurança, vamos tentar carregar o índice existente, mas com um aviso.
                # Ou, poderia definir recreate_index = False e sys.exit(1) se o documento fonte é crucial.
                print(f"Tentando carregar o índice FAISS existente de {FAISS_INDEX_PATH} apesar da ausência do documento original...")

            current_doc_timestamp = os.path.getmtime(DOCUMENT_PATH) if os.path.exists(DOCUMENT_PATH) else None

            if saved_doc_timestamp and current_doc_timestamp and saved_doc_timestamp == current_doc_timestamp:
                print(f"Carregando índice FAISS existente de {FAISS_INDEX_PATH} (documento não modificado).")
                vectorstore = FAISS.load_local(
                    FAISS_INDEX_PATH,
                    embeddings,
                    allow_dangerous_deserialization=True
                )
                recreate_index = False # Não precisa recriar
                print("Índice FAISS carregado com sucesso.")
            elif not current_doc_timestamp and saved_doc_timestamp : # Documento original sumiu
                 print(f"AVISO: Documento original {DOCUMENT_PATH} não encontrado, mas um índice existe.")
                 print(f"Carregando índice FAISS existente de {FAISS_INDEX_PATH}. Pode estar desatualizado.")
                 vectorstore = FAISS.load_local(
                    FAISS_INDEX_PATH,
                    embeddings,
                    allow_dangerous_deserialization=True
                )
                 recreate_index = False # Não precisa recriar
                 print("Índice FAISS carregado com sucesso (potencialmente desatualizado).")
            else:
                print("Documento modificado ou timestamp inválido. Recriando o índice...")
                if os.path.exists(FAISS_INDEX_PATH):
                    shutil.rmtree(FAISS_INDEX_PATH) #Usa shutil.rmtree para diretórios
        except FileNotFoundError:
             print(f"Arquivo {DOCUMENT_PATH} não encontrado durante a verificação de timestamp. Necessário recriar índice.")
             if os.path.exists(FAISS_INDEX_PATH):
                shutil.rmtree(FAISS_INDEX_PATH)
        except Exception as e:
            print(f"Erro ao carregar/verificar índice FAISS ou metadados: {e}. Recriando o índice...")
            if os.path.exists(FAISS_INDEX_PATH):
                shutil.rmtree(FAISS_INDEX_PATH) #Usa shutil.rmtree

    elif os.path.exists(FAISS_INDEX_PATH): # Existe o índice mas não os metadados
        print(f"Índice FAISS encontrado em {FAISS_INDEX_PATH}, mas sem metadados de timestamp. Recriando por segurança.")
        shutil.rmtree(FAISS_INDEX_PATH) # Apaga para garantir a recriação com metadados
        recreate_index = True
    else: # Nem índice nem metadados existem
        print(f"Nenhum índice FAISS ou metadados encontrados. Criando um novo...")
        recreate_index = True


    # --- CRIAÇÃO DO ÍNDICE (SE NECESSÁRIO) ---
    if recreate_index:
        print(f"Iniciando processo de criação de novo índice a partir de {DOCUMENT_PATH}...")
        # 1. Carregar Documentos
        print(f"Carregando documento: {DOCUMENT_PATH}")
        try:
            if not os.path.exists(DOCUMENT_PATH):
                print(f"Erro Crítico: O documento {DOCUMENT_PATH} não foi encontrado e é necessário para criar o índice.")
                sys.exit(1)
            loader = TextLoader(DOCUMENT_PATH, encoding='utf-8')
            docs = loader.load()
            if not docs:
                print(f"Erro: O documento {DOCUMENT_PATH} está vazio ou não pôde ser carregado.")
                sys.exit(1)
            print(f"Carregado {len(docs)} seção(ões) do documento.")
        except Exception as e:
            print(f"Erro ao carregar o documento {DOCUMENT_PATH}: {e}")
            sys.exit(1)

        # 2. Dividir Documentos em Chunks
        print(f"Dividindo documento em chunks (tamanho={CHUNK_SIZE}, sobreposição={CHUNK_OVERLAP})...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        splits = text_splitter.split_documents(docs)
        if not splits:
            print("Erro: Nenhum chunk foi criado. Verifique o conteúdo do documento e o tamanho do chunk.")
            sys.exit(1)
        print(f"Documento dividido em {len(splits)} chunks.")

        # 3. Criação do Vector Store
        print("Criando novo vector store FAISS...")
        try:
            vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
            print("Vector store FAISS criado com sucesso.")

            #Salvar o índice e os metadados (timestamp) ###
            # Garante que o diretório base do índice exista
            if not os.path.exists(FAISS_INDEX_PATH):
                os.makedirs(FAISS_INDEX_PATH)
            vectorstore.save_local(FAISS_INDEX_PATH)
            print(f"Índice FAISS salvo em {FAISS_INDEX_PATH}")

            # Salva o timestamp do documento atual nos metadados
            try:
                current_doc_timestamp_at_creation = os.path.getmtime(DOCUMENT_PATH)
                with open(INDEX_METADATA_PATH, 'wb') as f:
                    pickle.dump({'doc_timestamp': current_doc_timestamp_at_creation}, f)
                print(f"Timestamp do documento salvo em {INDEX_METADATA_PATH}")
            except Exception as e:
                print(f"Erro ao salvar timestamp do documento: {e}")
        except Exception as e:
            print(f"Erro ao criar ou salvar o vector store FAISS: {e}")
            sys.exit(1)

    # --- 4. Criar Retriever ---
    if vectorstore is None:
        print("Erro crítico: Vectorstore não foi carregado nem criado. Saindo.")
        sys.exit(1)
    retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
    print("Retriever criado (recuperando k=3 documentos).")

    # --- 5. Definir Template do Prompt RAG ---
    template = """Você é um assistente especializado em procedimentos internos de uma Cooperativa, com foco em sistemas bancários como INSS e Débito Automático. Sua função é ajudar o usuário a entender esses procedimentos.
    Use o 'Contexto' fornecido abaixo como sua *principal fonte de informação* sempre que ele for relevante para responder a pergunta.
    Se a resposta *não estiver explicitamente no 'Contexto'*, mas a pergunta for sobre procedimentos internos, sistemas bancários (INSS, Débito Automático) ou assuntos *diretamente relacionados* a esses tópicos, você *pode usar seu conhecimento geral* sobre esses domínios para formular a resposta.
    Em caso de qualquer conflito entre seu conhecimento geral e o 'Contexto', **priorize e confie SEMPRE no 'Contexto'**. A informação interna é a fonte final.
    Mantenha a resposta *clara, concisa e diretamente relacionada* à pergunta do usuário. Evite prolixidade.
    Sempre em português.

    Contexto:
    {context}

    Pergunta:
    {question}

    Resposta Concisa:"""
    prompt = ChatPromptTemplate.from_template(template)
    print("Template de prompt RAG definido.")

    # --- 6. Inicializar LLM ---
    print(f"Inicializando LLM Ollama com o modelo: {LLM_MODEL_NAME}...")
    try:
        llm = OllamaLLM(model=LLM_MODEL_NAME)
         # Teste rápido para verificar se o LLM está acessível
        _ = llm.invoke("Teste de conexão com LLM.")
        print("LLM Ollama inicializado com sucesso.")
    except Exception as e:
        print(f"\nErro ao inicializar LLM Ollama: {e}")
        print(f"Verifique se o serviço Ollama está rodando e se o modelo '{LLM_MODEL_NAME}' foi baixado.")
        sys.exit(1)

    # --- 7. Criar a Cadeia (Chain) RAG ---
    # Função auxiliar para formatar os documentos recuperados para o prompt
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Define a cadeia RAG usando a sintaxe LangChain Expression Language (LCEL)
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    print("Cadeia RAG criada e pronta para uso.")

    # --- 8. Consultar a Cadeia RAG Interativamente ---
    print("\n--- Sistema Pronto para Consultas ---")
    while True:
        try:
            user_question = input("Faça uma pergunta sobre convênio (ou digite 'sair' para terminar): ")
            if user_question.lower() == 'sair':
                break
            if not user_question.strip():
                continue

            print("\nProcessando...")
            # Invoca a cadeia RAG com a pergunta do usuário
            response = rag_chain.invoke(user_question)

            print("\nResposta:")
            print(response)
            print("\n" + "="*60 + "\n")

        except KeyboardInterrupt:
            print("\nSaindo...")
            break
        except Exception as e:
            print(f"\nOcorreu um erro durante a consulta: {e}")
            print("Tente novamente ou digite 'sair'.")

if __name__ == "__main__":
    run_rag_query()
    print("Aplicação encerrada.")

# ```
# **Explicação do Código:**
# *   **Carregamento (`TextLoader`):** Lê o arquivo de texto preparado.[44, 45]
# *   **Divisão (`RecursiveCharacterTextSplitter`):** Quebra o texto em chunks.[31, 43, 44, 45]
# *   **Embedding (`OllamaEmbeddings`):** Conecta-se ao Ollama para usar o modelo de embedding especificado (`nomic-embed-text`) para converter chunks em vetores.[24, 45, 48, 49, 51, 57, 58]
# *   **Armazenamento (`FAISS.from_documents`):** Cria o índice FAISS em memória com os chunks e seus embeddings.[31, 36, 50, 51, 52, 53, 58] (O código comentado mostra como salvar/carregar localmente para persistência).
# *   **Recuperação (`vectorstore.as_retriever`):** Cria um objeto que sabe como buscar chunks relevantes no FAISS.[43, 44, 49, 52, 54]
# *   **Prompt (`ChatPromptTemplate`):** Define a estrutura da entrada que será enviada ao LLM, incluindo placeholders para o contexto recuperado (`{context}`) e a pergunta do usuário (`{question}`).[43, 45, 51, 52, 54, 58, 61, 62, 63]
# *   **Geração (`OllamaLLM`):** Conecta-se ao Ollama para usar o modelo LLM especificado (`gemma3:4b`) para gerar a resposta.[43, 45, 50, 51, 52, 54, 58, 62]
# *   **Cadeia (`|` - LCEL):** A LangChain Expression Language (LCEL) permite encadear os passos: pegar a pergunta, passá-la ao retriever para obter o contexto, formatar o contexto, passar tudo para o prompt, enviar ao LLM e parsear a saída como string.[44, 45, 49, 54]

# Implementar RAG, mesmo de forma simplificada, envolve a orquestração de vários componentes. LangChain abstrai parte dessa complexidade, mas introduz seus próprios conceitos (Vector Stores, Retrievers, Chains/Runnables) que são novos para iniciantes. O script fornecido visa ser um ponto de partida claro e funcional.


Inicializando embeddings Ollama com o modelo: nomic-embed-text...
Embeddings Ollama inicializados com sucesso.
Verificando metadados do índice em faiss_index_procedures/index_metadata.pkl...
Carregando índice FAISS existente de faiss_index_procedures (documento não modificado).
Índice FAISS carregado com sucesso.
Retriever criado (recuperando k=3 documentos).
Template de prompt RAG definido.
Inicializando LLM Ollama com o modelo: gemma3:4b...
LLM Ollama inicializado com sucesso.
Cadeia RAG criada e pronta para uso.

--- Sistema Pronto para Consultas ---
Faça uma pergunta sobre convênio (ou digite 'sair' para terminar): quem é diego?

Processando...

Resposta:
Diego é o coordenador da equipes de convenios.


Faça uma pergunta sobre convênio (ou digite 'sair' para terminar): temo algum procedimento que reprocessa os debitos?

Processando...

Resposta:
Sim, existe o procedimento 7242: "REPROCESSA ARQUIVOS DEBITO AUTOMATICO."


Faça uma pergunta sobre convênio (ou digite 'sair' para termin